In [1]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Usual library
import itertools
import numpy             as np
import pandas            as pd

# root_numpy (http://scikit-hep.org/root_numpy)
from root_numpy import root2array

Welcome to JupyROOT 6.10/02


# Data and functions to be optimized

In [2]:
ar=root2array('410000.root','nominal_Loose',
               branches=['jet_pt','jet_eta','jet_phi','jet_mv2c10','jet_isbtagged_77',
                'el_pt','el_eta','el_phi',
                'mu_pt','mu_eta','mu_phi',
                'mu']
             ).view(np.recarray)
data=pd.DataFrame(ar)
data.head(10)
len(data)

1622880

In [3]:
large_data = pd.concat([data for _ in np.arange(1)])
len(large_data)

1622880

In [4]:
def square_array(a):
    import numpy as np
    lens = np.array([len(i) for i in a])
    mask = np.arange(lens.max()) < lens[:,None]    
    out = np.zeros(mask.shape, dtype='float32')
    out.fill(np.nan)    
    out[mask] = np.concatenate(a)   
    return out

def all_pairs(a,b=None,Nmax=None,axis=1):  
    import numpy as np
    import itertools   
    same_arrays = b is None
    if not same_arrays:
        good_shape=np.array_equal(np.delete(a.shape,axis),np.delete(b.shape,axis))
        if not good_shape:
            err  = 'The shape along all dimensions but the one of axis={}'.format(axis)
            err += ' should be equal, while here:\n'
            err += '  -> shape of a is {} \n'.format(a.shape)
            err += '  -> shape of b is {} \n'.format(b.shape)
            raise NameError(err)           
    if Nmax:
        sl=[slice(None)]*a.ndim
        sl[axis]=slice(0,Nmax)
        if same_arrays: a,b=a[sl],None
        else          : a,b=a[sl],b[sl]          
    if same_arrays:
        ia,jb=np.arange(a.shape[axis]),[]
    else:
        ia,jb=np.arange(a.shape[axis]),np.arange(b.shape[axis])          
    dt=np.dtype([('', np.intp)]*2)
    if same_arrays: ij=np.fromiter(itertools.combinations(ia,2),dtype=dt)     
    else          : ij=np.fromiter(itertools.product(ia,jb),dtype=dt)
    ij=ij.view(np.intp).reshape(-1, 2)
    if same_arrays: out=np.take(a,ij,axis=axis)
    else          : out=np.stack([a.take(ij[:,0],axis=axis),b.take(ij[:,1],axis=axis)],axis=axis+1)
    return out

# Helper function for parallelization

In [5]:
def adjust_array_size(small_array_list):
    Nmax=np.max([a.shape[1] for a in small_array_list])
    res=[]
    for a in small_array_list:
        tmp=np.zeros((a.shape[0],Nmax))
        tmp.fill(np.nan)
        tmp[:,:a.shape[1]:] = a
        res.append(tmp)
    return res

# Reference arrays and time

In [6]:
%timeit square_array(large_data['jet_pt'].values)
%timeit square_array(large_data['el_pt'].values)

1 loop, best of 3: 897 ms per loop
1 loop, best of 3: 864 ms per loop


In [7]:
jet_pt_ref=square_array(large_data['jet_pt'].values)
ele_pt_ref=square_array(large_data['el_pt'].values)
print(jet_pt_ref.shape)
print(ele_pt_ref.shape)

(1622880, 13)
(1622880, 3)


# Try with `ipyparallel`

In [8]:
import ipyparallel as ipp
rc=ipp.Client()
view=rc[:]

def just_try_ipypar():
    print(view,rc.ids)
    view.block=True
    view['a'] = 5
    view['b'] = 10
    view.apply(lambda x: a+b+x, 27)

In [9]:
just_try_ipypar()

(<DirectView [0, 1, 2, 3,...]>, [0, 1, 2, 3, 4, 5])


## Making the squared array

In [10]:
def ipypar_square_array(a,Nworkers):
    split_arrays=np.split(a,Nworkers)
    outputs=adjust_array_size(view.map(square_array,split_arrays))
    return np.concatenate(outputs)

In [12]:
%timeit square_array(large_data['jet_pt'].values)
%timeit ipypar_square_array(large_data['jet_pt'].values,6)

1 loop, best of 3: 880 ms per loop
1 loop, best of 3: 9.05 s per loop


In [13]:
r=ipypar_square_array(large_data['jet_pt'].values,6)
print(r.shape)

(1622880, 13)


## Making all the pairs

In [14]:
def ipypar_all_pairs(a,b,Nworkers):
    split_a,split_b=np.split(a,Nworkers),np.split(b,Nworkers)
    outputs=view.map(all_pairs,split_a,split_b)
    return np.concatenate(outputs)

In [15]:
%timeit all_pairs(jet_pt_ref,ele_pt_ref)
%timeit ipypar_all_pairs(jet_pt_ref,ele_pt_ref,6)

1 loop, best of 3: 606 ms per loop
1 loop, best of 3: 1.19 s per loop


In [16]:
r=ipypar_all_pairs(jet_pt_ref,ele_pt_ref,6)
print(r.shape)

(1622880, 39, 2)


# Try with `concurrent.futures`

## Function definition

In [17]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

def multithreading(func, args, workers):
    with ThreadPoolExecutor(max_workers=workers) as executor:
        exec_res = executor.map(func,args)
    return list(exec_res)

def multiprocessing(func, args, workers):
    with ProcessPoolExecutor(max_workers=workers) as executor:
        exec_res = executor.map(func,args)
    return list(exec_res)

## Making the squared array

In [20]:
Nwokers=6
split_arrays=np.split(large_data['jet_pt'].values,Nwokers)
%timeit square_array(large_data['jet_pt'].values)
%timeit multithreading (func=square_array,args=split_arrays,workers=Nwokers)
%timeit multiprocessing(func=square_array,args=split_arrays,workers=Nwokers)

1 loop, best of 3: 905 ms per loop
1 loop, best of 3: 1.64 s per loop
1 loop, best of 3: 10.2 s per loop


In [21]:
r=multithreading (func=square_array,args=split_arrays,workers=Nwokers)
print([i.shape for i in r])

[(270480, 11), (270480, 13), (270480, 11), (270480, 13), (270480, 12), (270480, 12)]


## Making all the possible pairs

In [30]:
def all_pair_one_arg(doublet):
    return all_pairs(doublet[0],doublet[1])

# Get longer arrays, just for this test.
jet_pt_ref_long=np.repeat(jet_pt_ref,5,axis=0)
ele_pt_ref_long=np.repeat(ele_pt_ref,5,axis=0)

Nworkers=6
split_pair=[[a,b] for a,b in zip(np.split(jet_pt_ref_long,Nwokers),np.split(ele_pt_ref_long,Nwokers))]
%timeit all_pairs(jet_pt_ref_long,ele_pt_ref_long)
%timeit np.concatenate(multithreading(all_pair_one_arg,split_pair,workers=Nworkers))

1 loop, best of 3: 3.1 s per loop
1 loop, best of 3: 2.37 s per loop


# Try with `threading`

## Required functions

In [23]:
from threading import Thread

class p_square_array(Thread):    
    def __init__(self,a):
        Thread.__init__(self)
        self.data=a
        self.output=None       
    def run(self):
        self.output=square_array(self.data)
        
class p_all_pairs(Thread):    
    def __init__(self,a,b):
        Thread.__init__(self)
        self.a=a
        self.b=b
        self.output=None        
    def run(self,):
         self.output=all_pairs(a=self.a,b=self.b)

## Making square arrays

In [24]:
def threading_square_array(data,Nworkers):
    split_arrays=np.split(data,Nwokers)
    all_threads=[p_square_array(a) for a in split_arrays]
    for t in all_threads: t.start()
    for t in all_threads: t.join()
    
    output_array = [t.output for t in all_threads]
    Nmax=np.max([a.shape[1] for a in output_array])
    res=[]
    for a in output_array:
        tmp=np.zeros((a.shape[0],Nmax))
        tmp.fill(np.nan)
        tmp[:,:a.shape[1]:] = a
        res.append(tmp)
        
    return np.concatenate(res)

In [25]:
%timeit threading_square_array(large_data['jet_pt'].values,8)
res=threading_square_array(large_data['jet_pt'].values,8)
print(res.shape)

1 loop, best of 3: 1.84 s per loop
(1622880, 13)


In [26]:
def p_load_pts():
    jet_pt=threading_square_array(large_data['jet_pt'].values,8)
    ele_pt=threading_square_array(large_data['el_pt'].values,8)
    mu_pt =threading_square_array(large_data['mu_pt'].values,8)

def load_pts():
    jet_pt=square_array(large_data['jet_pt'].values)
    ele_pt=square_array(large_data['el_pt'].values)
    mu_pt =square_array(large_data['mu_pt'].values)

%timeit p_load_pts()
%timeit load_pts()

1 loop, best of 3: 5.12 s per loop
1 loop, best of 3: 2.71 s per loop


## Making all possible pairs

In [27]:
def threading_all_pairs(a,b,Nworkers):
    split_arrays_a=np.split(a,Nwokers)
    split_arrays_b=np.split(b,Nwokers)
    all_threads=[p_all_pairs(a,b) for a,b in zip(split_arrays_a,split_arrays_b)]
    for t in all_threads: t.start()
    for t in all_threads: t.join()
    return np.concatenate([t.output for t in all_threads])

In [28]:
%timeit threading_all_pairs(jet_pt_ref,ele_pt_ref,8)
r=threading_all_pairs(jet_pt_ref,ele_pt_ref,8)
print(r.shape)

1 loop, best of 3: 476 ms per loop
(1622880, 39, 2)


In [29]:
%timeit all_pairs(jet_pt_ref,ele_pt_ref)
r=all_pairs(jet_pt_ref,ele_pt_ref)
print(r.shape)

1 loop, best of 3: 655 ms per loop
(1622880, 39, 2)
